Note that the main codes and comments on data are adapted from a notebook from Hackweek tutorial.
The link is https://github.com/ICESAT-2HackWeek/2020_ICESat-2_Hackweek_Tutorials.git.

### 1. Libraries import
Import all the necessary libraries.

In [1]:
# import requests
# import getpass
# import socket
# import json
# import zipfile
# import math
# import pprint
# import time

# utility modules
import glob
import os
import sys
import re
import io

# the usual suspects
import numpy as np
import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.img_tiles as cimgt
import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# modules you'll need if you're downloading the data
from icepyx import icesat2data as ipd
import shutil
import geopandas as gpd

import fiona
import pyproj
import h5py

# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

### 2. Plot ICESat-2 data
First plot background map layers.

In [2]:
extent = [88.1, 89, 27.7, 28.5]
x = [88.1443, 88.2269, 88.3643, 88.2708, 88.4037, 88.4865, 88.625, 88.5314, 88.6636, 88.7459, 88.8852, 88.7912, 88.9223, 89.0054]
y = [27.6996, 28.5008, 27.6990, 28.5008, 27.6983, 28.5008, 27.6997, 28.5008, 27.6999, 28.5010, 27.6965, 28.5012, 27.6961, 28.5012]

request = cimgt.OSM()

fig = plt.figure(figsize=(9, 13))
ax = plt.axes(projection=request.crs)

ax.plot(x[0:2], y[0:2], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[2:4], y[2:4], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[4:6], y[4:6], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[6:8], y[6:8], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[8:10], y[8:10], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[10:12], y[10:12], transform=ccrs.PlateCarree(), color='red')
ax.plot(x[12:14], y[12:14], transform=ccrs.PlateCarree(), color='red')

gl = ax.gridlines(draw_labels=True, alpha=0.2)
# gl.xlabels_top = gl.ylabels_right = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax.set_extent(extent)
ax.add_image(request, 10)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
fiona.drvsupport.supported_drivers['kml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['KML'] = 'rw' # enable KML support which is disabled by default

# kml data
lake_filepath = os.getcwd()+'/sikkimlakes.kml'
gdfl = gpd.read_file(lake_filepath) #GeoDataFrame object
glacier_filepath = os.getcwd()+'/sikkimglaciers.kml'
gdfg = gpd.read_file(glacier_filepath) #GeoDataFrame object
border_filepath = os.getcwd()+'/borders.kml'
gdfb = gpd.read_file(border_filepath)

# trajectory data  x lon        y lat
x = [88.1443, 88.2269, 88.3643, 88.2708, 88.4037, 88.4865, 88.625, 88.5314, 88.6636, 88.7459, 88.8852, 88.7912, 88.9223, 89.0054]
y = [27.6996, 28.5008, 27.6990, 28.5008, 27.6983, 28.5008, 27.6997, 28.5008, 27.6999, 28.5010, 27.6965, 28.5012, 27.6961, 28.5012]


# Overlay lake outline
f, ax = plt.subplots(1, figsize=(9, 13))
plt.title('ICESat-2 trajectory in Sikkim')
gdfb.plot(ax=ax, facecolor='white', edgecolor='black')
gdfl.plot(ax=ax, facecolor='lightskyblue', edgecolor='gray')
gdfg.plot(ax=ax, facecolor='blue', edgecolor='white')

plt.plot(x[0:2], y[0:2], color='red', label='477')
plt.plot(x[2:4], y[2:4], color='olivedrab', label='241')
plt.plot(x[4:6], y[4:6], color='gold', label='919')
plt.plot(x[6:8], y[6:8], color='mediumslateblue', label='683')
plt.plot(x[8:10], y[8:10], color='mediumspringgreen', label='1361')
plt.plot(x[10:12], y[10:12], color='violet', label='1125')
plt.plot(x[12:14], y[12:14], color='moccasin', label='416')
plt.annotate("Como Chamling", (88.19, 28.4))
plt.legend(loc='upper right',title='track ID')
ax.set_ylim([27.7,28.5]) #adjust to frame glacier
ax.set_xlim([88.1,89]);
plt.xlabel('longitude')
plt.ylabel('latitude')

plt.show()


# image = mpimg.imread("January_original.tif")
# plt.imshow(image)
# plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

pre read data ATL03 ATL06 and ATL13    Reference Ground Track (RGT)

In [4]:
sys.path.append(os.path.join(os.getcwd(), '..'))
from readers.read_HDF5_ATL03 import read_HDF5_ATL03
from readers.get_ATL03_x_atc import get_ATL03_x_atc

# read the IS2 data with Tyler's ATL03 reader:
ATL03_file='ATL03_20190128163325_04770206_003_01.h5'
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)
# add x_atc to the ATL03 data structure (this function adds to the LS2_ATL03_mds dictionary)
get_ATL03_x_atc(IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams)

In [5]:
beam='gt2l'
rgt="0477"
cycle="02"
# select the 2l beam from ATL03
D3 = IS2_atl03_mds[beam]

# create scatter plot of photon data (e.g., photon elevation vs x_atc)
fig=plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
ax.plot(D3['heights']['x_atc'], D3['heights']['h_ph'],'k.',markersize=0.25, label='all photons')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
h_leg=ax.legend()

ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
# print(type(D3['heights']))
# print(D3['heights'].keys())

In [7]:
# from readers.read_HDF5_ATL13 import read_HDF5_ATL13

# # read the IS2 data with ATL13 reader:
# ATL13_file='ATL13_20190128160153_04770201_003_01.h5'
# IS2_atl13_mds, IS2_atl13_attrs, IS2_atl13_beams =read_HDF5_ATL13(ATL13_file)
# D13 = IS2_atl13_mds[beam]

In [8]:
# print(D13['ht_water_surf'])
# print(D13['ht_ortho'])
# print(D13['ht_water_surf']-D13['ht_ortho'])

ATL03 can show where the surface is, but it doesn't explicitly report a surface height. It's also a bulky product that can be slow to load. ATL06 reduces elevation data to a 20-meter posting, and gives one elevation per 20 meters (instead of hundreds in ATL03).
To help work with ATL06 data, we'll use a simple piece of code that reads ATL06 data one beam at a time and stores it in a dictionary. The code has a default set of fields to be read from an ATL06 file, and stores the data from each field in the output dictionary. It also takes care of removing bad data, by setting values that are marked as invalid in the file to NaN.

In [9]:
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

We'll use this to read the ATL06 data that correspond to the ATL03 data in the previous cell, and will plot the ATL06 elevations on top of the ATL03 photons.

In [10]:
# find the matching ATL06 file
ATL06_file='ATL06_20190128163325_04770206_003_01.h5'
D6=atl06_to_dict(ATL06_file,'/gt2l', index=None, epsg=3031)

# plot the elevations on top of the previous axes.  You should be able to scroll up to the previous plot and see the ATL06 points.
ax.plot(D6['x_atc'], D6['h_li'],'r.', label='ATL06')
ax.legend();

In [11]:
{field:D6[field].size for field in ['x_atc','h_li']}

{'x_atc': 83928, 'h_li': 83928}

In [12]:
#select the 2l beam from ATL03
D3 = IS2_atl03_mds[beam]

# find the matching ATL06 file
D6=atl06_to_dict(ATL06_file, beam, index=None, epsg=3031)
# create scatter plot of photon data (e.g., photon elevation vs x_atc)
%matplotlib widget
f1,ax = plt.subplots(num=1,figsize=(6,4))
TEP=(D3['heights']['signal_conf_ph'][:,3] <-1)   
ax.plot(D3['heights']['x_atc'][TEP], D3['heights']['h_ph'][TEP],'b.',markersize=0.25, label='TEP')
BG=(D3['heights']['signal_conf_ph'][:,3] ==0)   |  (D3['heights']['signal_conf_ph'][:,3] ==1)
ax.plot(D3['heights']['x_atc'][BG], D3['heights']['h_ph'][BG],'k.',markersize=0.25, label='Background')
LMH=D3['heights']['signal_conf_ph'][:,3] >= 2
ax.plot(D3['heights']['x_atc'][LMH], D3['heights']['h_ph'][LMH],'g.',markersize=0.5, label='flagged photons')
ax.plot(D6['x_atc'], D6['h_li'],'r.', label='ATL06')
h_leg=ax.legend()

plt.title(beam+':'+os.path.basename(ATL03_file))

ax.set_xlabel('x_atc, m')
ax.set_ylabel('h, m')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The blue stripe is signals flagged with flag which indicates that nobody should believe that this is the surface. That is what we call the transmitter echo path arrival and these photons were fired into an optical fiber onboard the spacecraft. The photons rattled around this optical fiber for a while and came back to the detectors. 
TEP = Transmitter Echo Pulse
Now the window of photons around the surface is much more scattered, there are some places where ATL06 is reporting heights for something that's probably not the surface, and there's a stripe of extra photons below the surface that doesn't make any obvious sense.  The new (blue) stripe of photons is the TEP or Transmitter Echo Pulse, which records a packet of photons that are looped from the transmit to the receive side of ATLAS to help monitor its impulse response.  ATL06 ignores these photons, and we should too.  

In [13]:
print(D6['beam'])

gt2l


In [14]:
from readers.read_HDF5_ATL13 import read_HDF5_ATL13

# read the IS2 data with ATL13 reader:
ATL13_file='ATL13_20190128160153_04770201_003_01.h5'
IS2_atl13_mds, IS2_atl13_attrs, IS2_atl13_beams =read_HDF5_ATL13(ATL13_file)

Now deal with ATL06 and ATL13 to produce glacier thicknesses and lake surface heights.
Start of the data process.

In [ ]:
IS2_atl03_mds, IS2_atl03_attrs, IS2_atl03_beams =read_HDF5_ATL03(ATL03_file)
D6=atl06_to_dict(ATL06_file, beam, index=None, epsg=3031)